In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
def login(login, psswd, driver):
    driver.get("https://reservation.42network.org/signin")
    login_field = driver.find_element_by_id("user_login")
    psswd_field = driver.find_element_by_id("user_password")
    login_field.clear()
    login_field.send_keys(login)
    psswd_field.clear()
    psswd_field.send_keys(psswd)
    psswd_field.send_keys(Keys.RETURN)

In [3]:
def logout(driver):
    driver.find_elemt_by_xpath("//*[@id='navbar-main']/div[2]/a[2]").click()
    driver.find_element_by_xpath("//span[text()='logout']").click()    

In [8]:
def get_week_days(driver):
    return driver.find_elements_by_xpath("//div[@class='fc-content-skeleton']//div[@class='fc-event-container']")

In [9]:
def get_elements_in_day(day_element):
    return day_element.find_elements_by_xpath(".//a")

In [7]:
slot_times = ['08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00']

In [6]:
# driver = webdriver.Chrome("./chromedriver_linux")
driver = webdriver.Firefox()
login("jbarment", "69@TheEelHouse!", driver)

In [10]:
weekdays = get_week_days(driver)
mday = weekdays[6]

In [14]:
for el in mday.find_elements_by_xpath(".//a/div/div[@data-start='13:00']/../.."):
    print(el.get_attribute("class"))

fc-time-grid-event fc-event fc-start fc-end
fc-time-grid-event fc-event fc-start fc-end fc-time-grid-event-inset
fc-time-grid-event fc-event fc-start fc-end fc-time-grid-event-inset


In [34]:
sloto = mday.find_elements_by_xpath(".//a/div/div[@data-start='13:00']/../..")[0]

In [39]:
text = sloto.find_element_by_class_name("fc-title").text

In [33]:
class slot():
    def __init__(self, day, hour: str, reserved = False, desired = True, E1 = None, E2 = None, TDM = None, reserved_floor = None, date = None):
        self.hour = hour
        self.buttons = {'E1': E1, 'E2': E2, 'TDM': TDM}
        self.reserved = reserved
        self.reserved_floor = reserved_floor
        self.desired = desired
        self.date = date
    

SyntaxError: unexpected EOF while parsing (<ipython-input-33-54afbd2b5956>, line 11)

In [15]:
el = mday.find_elements_by_xpath(".//a/div/div[@data-start='13:00']/../..")[0]

In [25]:
class wanted_slots():
    def __init__(self, monday_start = None, monday_end = None, tuesday_start = None, tuesday_end = None, wednesday_start = None, wednesday_end = None, thursday_start = None,
                 thursday_end = None, friday_start = None, friday_end = None, saturday_start = None, saturday_end = None, sunday_start = None, sunday_end = None):
        self.monday_start = monday_start
        self.monday_end = monday_end
        self.tuesday_start = tuesday_start
        self.tuesday_end = tuesday_end
        self.wednesday_start = wednesday_start
        self.wednesday_end = wednesday_end
        self.thursday_start = thursday_start
        self.thursday_end = thursday_end
        self.friday_start = friday_start
        self.friday_end = friday_end
        self.saturday_start = saturday_start
        self.saturday_end = saturday_end
        self.sunday_start = sunday_start
        self.sunday_end = sunday_end
        self.array = [[monday_start, monday_end], [tuesday_start, tuesday_end], [wednesday_start, wednesday_end], [thursday_start, thursday_end], [friday_start, friday_end], [saturday_start, saturday_end], [sunday_start, sunday_end]]
        
class agenda():
    def __init__(self, wanted_slots):
        self.days = [[],[],[],[],[],[],[]]
        for day in range(7):
            print(day)